In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df_orders = pd.read_csv('/Users/patchong/Google Drive/My Drive/Work/Prep/Challenges/Curated/THA/Items - Dec 2022.csv')
df_act = pd.read_csv('/Users/patchong/Google Drive/My Drive/Work/Prep/Challenges/Curated/THA/Touchstream - 2022.csv')

In [3]:
# # splitting dataframe by row index
# df1 = df_act.iloc[:600000]
# df2 = df_act.iloc[600000:1200000]
# df3 = df_act.iloc[1200000:1800000]
# df4 = df_act.iloc[1800000:2400000]
# df5 = df_act.iloc[2400000:3000000]
# df6 = df_act.iloc[3000000:]
# print("Shape of new dataframes - {} , {}, {}, {}, {}, {}".format(df1.shape, df2.shape, df3.shape, df4.shape, df5.shape, df6.shape))

In [4]:
# df1.to_csv('Touchstream - 2022-1.csv')
# df2.to_csv('Touchstream - 2022-2.csv')
# df3.to_csv('Touchstream - 2022-3.csv')
# df4.to_csv('Touchstream - 2022-4.csv')
# df5.to_csv('Touchstream - 2022-5.csv')
# df6.to_csv('Touchstream - 2022-6.csv')

In [5]:
# Note, there are cases where a user is not in ops but there is an odd user_id.
df_act = df_act.dropna(subset=['fact_attributions_user_in_ops'])
df_act.isna().sum()

fact_attributions_user_in_ops              0
fact_attributions_user_id                  0
fact_attributions_touch_source             0
fact_attributions_source_group           476
fact_attributions_touch_occurred_date      0
dtype: int64

In [6]:
# # user with order
user_id = 'urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b' 
# # user with no orders
# user_id = 'urn:user:d5009cdc-54c0-441d-ac9a-62e7d77a20e3' 

# # user_id = ['urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b', 'urn:user:d5009cdc-54c0-441d-ac9a-62e7d77a20e3']

df_act = df_act[df_act.fact_attributions_user_id == user_id]
df_orders = df_orders[df_orders.dim_users_user_id == user_id]
# df_act = df_act[df_act.fact_attributions_user_id.isin(user_id)]
# df_orders = df_orders[df_orders.dim_users_user_id.isin(user_id)]

In [7]:
# left join so can calculate conversion (i.e. users who have activity but no orders)
merged = pd.merge(df_act, df_orders, left_on='fact_attributions_user_id', right_on='dim_users_user_id', how='left')

In [8]:
print(merged.shape)
merged.sort_values('fact_attributions_touch_occurred_date', ascending=False).head()

(2980, 14)


,fact_attributions_user_in_ops,fact_attributions_user_id,fact_attributions_touch_source,fact_attributions_source_group,fact_attributions_touch_occurred_date,fact_purchased_orders_link_to_ops,fact_purchased_orders_exposed_order_id,fact_purchased_orders_order_id,dim_users_link_to_ops,dim_users_user_id,fact_purchased_orders_purchased_date,dim_users_created_date,fact_purchased_items_total_base_revenue_excluding_refund,fact_purchased_items_total_estimated_cost
182,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,direct,direct,2022-12-31,https://ops.curated.com/orders/AgAAACkAV4qg-z5XS5Scnp8xxmjSpg,88733-40119,urn:order:578aa0fb-3e57-4b94-9c9e-9f31c668d2a6,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,2022-12-16,2017-11-27,479.97,359.96
175,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,trigger,crm,2022-12-31,https://ops.curated.com/orders/AgAAACkAiB44JYfKQJeYuuATACDWXg,43463-25847,urn:order:881e3825-87ca-4097-98ba-e0130020d65e,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,2022-11-30,2017-11-27,377.97,348.68
177,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,trigger,crm,2022-12-31,https://ops.curated.com/orders/AgAAACkA_kFraIM8SGGA8qGoUIMSWQ,43914-86010,urn:order:fe416b68-833c-4861-80f2-a1a850831259,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,2022-12-30,2017-11-27,329.97,467.46
178,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,trigger,crm,2022-12-31,https://ops.curated.com/orders/AgAAACkAKSukO82FTlu-ZJauf-ABxA,79248-34631,urn:order:292ba43b-cd85-4e5b-be64-96ae7fe001c4,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,2022-12-23,2017-11-27,299.99,228.79
179,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,trigger,crm,2022-12-31,https://ops.curated.com/orders/AgAAACkAv4_qSk-NTGW3sIvQVUcjiQ,01916-26351,urn:order:bf8fea4a-4f8d-4c65-b7b0-8bd055472389,https://ops.curated.com/consumers/AgAAAAEAxUPTmRNLR9CzsdmmTJeRKw,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,2022-12-30,2017-11-27,120.00,54.00


There are 5 cases
1.  users with no activity and orders (we don't care about these orders for an attribution model)
2.  users with activity but no orders
3.  users with activity and orders that are greater than activity date
4.  users with activity and orders BUT with activities with missing orders
5.  __users with activity and multiple orders on same day__ (bc we don't group by order_id we "undercount" paths)
  - since assuming that multiples orders on the same day can be attributed to same

In [9]:
# users with activty but no orders
no_order_df = merged[pd.isnull(merged['dim_users_user_id'])][['fact_attributions_user_id', 'fact_purchased_orders_order_id', 'fact_attributions_touch_source', 'fact_attributions_touch_occurred_date', 'fact_purchased_orders_purchased_date']]
# no_order_df = merged[pd.isnull(merged['dim_users_user_id'])][['fact_attributions_user_id', 'fact_attributions_touch_source', 'fact_attributions_touch_occurred_date', 'fact_purchased_orders_purchased_date']]
no_order_df['fact_purchased_orders_purchased_date'] = -1
no_order_df['fact_purchased_orders_order_id'] = -1

In [10]:
# # testing
# print(no_order_df.sort_values('fact_attributions_touch_occurred_date').shape)
# no_order_df.sort_values('fact_attributions_touch_occurred_date', ascending=False).head()

In [11]:
# users with activities and orders

# isValid is so we can get the respective minimum order date for each user activity such that the order date is greater than the activity date
# e.g. user journey = activtity_1 > order_1 > activity_2 > activity_3 > order_2
# in the example above, order_1 would be associated with activity_1 and order_2 would be associated with both activity_2/3
merged['isValid'] = merged['fact_attributions_touch_occurred_date'] <= merged['fact_purchased_orders_purchased_date']

orders_df = merged[merged.isValid].sort_values('fact_purchased_orders_purchased_date').groupby(['fact_attributions_user_id', 'fact_purchased_orders_order_id', 'fact_attributions_touch_source', 'fact_attributions_touch_occurred_date']).first()[[\
    'fact_purchased_orders_purchased_date']].reset_index()

# orders_df = merged[merged.isValid].sort_values('fact_purchased_orders_purchased_date').groupby(['fact_attributions_user_id', 'fact_attributions_touch_source', 'fact_attributions_touch_occurred_date']).first()[[\
#     'fact_purchased_orders_purchased_date']].reset_index()

In [12]:
# # testing
# print(orders_df.sort_values('fact_attributions_touch_occurred_date').shape)
# orders_df.sort_values('fact_attributions_touch_occurred_date', ascending=False).head()

In [13]:
# users with activity and orders AND activity with missing orders
merged['max_order_date'] = merged.groupby(['fact_attributions_user_id'])['fact_purchased_orders_purchased_date'].transform('max')
merged['missing_orders'] = merged['fact_attributions_touch_occurred_date'] > merged['max_order_date']

missing_order_df = merged[merged.missing_orders].sort_values('fact_purchased_orders_purchased_date').groupby(['fact_attributions_user_id', 'fact_attributions_touch_source','fact_attributions_touch_occurred_date']).first()[[\
    'fact_purchased_orders_purchased_date']].reset_index()

missing_order_df['fact_purchased_orders_purchased_date'] = -1
missing_order_df['fact_purchased_orders_order_id'] = -1

In [14]:
# # testing
# print(missing_order_df.sort_values('fact_attributions_touch_occurred_date').shape)
# missing_order_df.sort_values('fact_attributions_touch_occurred_date', ascending=False).head()

In [15]:
# # testing
# merged[['fact_attributions_user_id', 'fact_attributions_touch_source', 'fact_attributions_touch_occurred_date', 'fact_purchased_orders_purchased_date']]\
# .sort_values(['fact_attributions_touch_occurred_date', 'fact_purchased_orders_purchased_date'], ascending=[False, False]).head(20)

In [16]:
res_df = pd.concat([no_order_df, orders_df, missing_order_df])

In [17]:
print(res_df.shape)
res_df.sort_values('fact_attributions_touch_occurred_date').head()

(2896, 5)


,fact_attributions_user_id,fact_purchased_orders_order_id,fact_attributions_touch_source,fact_attributions_touch_occurred_date,fact_purchased_orders_purchased_date
154,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:292ba43b-cd85-4e5b-be64-96ae7fe001c4,trigger,2022-01-03,2022-12-23
2020,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:c6d86888-b0aa-4f4b-83ff-59c5f0626123,direct,2022-01-03,2022-12-21
1875,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:bf8fea4a-4f8d-4c65-b7b0-8bd055472389,trigger,2022-01-03,2022-12-30
2310,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:ce24cad6-b0f7-4ebc-addd-9c6eaaca529e,direct,2022-01-03,2022-12-23
1724,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:bf8fea4a-4f8d-4c65-b7b0-8bd055472389,direct,2022-01-03,2022-12-30


In [18]:
# df_paths = res_df.groupby(['fact_attributions_user_id', 'fact_purchased_orders_purchased_date'])['fact_attributions_touch_source'].aggregate(
#     lambda x: x.unique().tolist()).reset_index()

df_paths = res_df.groupby(['fact_attributions_user_id', 'fact_purchased_orders_order_id', 'fact_purchased_orders_purchased_date'])['fact_attributions_touch_source'].aggregate(
    lambda x: x.unique().tolist()).reset_index()

# df_last_interaction = df.drop_duplicates('fact_attributions_user_id', keep='last')[['fact_attributions_user_id', 'fact_purchased_orders_order_id', 'fact_purchased_orders_purchased_date']]
# df_paths = pd.merge(df_paths, df_last_interaction, how='left', on='fact_attributions_user_id')

def get_list(x):
    x['fact_attributions_touch_source'].insert(0, 'Start')
    x['fact_attributions_touch_source'].append('Null' if x.fact_purchased_orders_purchased_date == -1 else 'Conversion' )
    return x

In [19]:
df_paths.apply(lambda x: get_list(x), axis=1)

,fact_attributions_user_id,fact_purchased_orders_order_id,fact_purchased_orders_purchased_date,fact_attributions_touch_source
0,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,-1,-1,"[Start, direct, trigger, Null]"
1,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:292ba43b-cd85-4e5b-be64-96ae7fe001c4,2022-12-23,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"
2,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:45073a2b-532a-44f5-af04-50143c27080f,2022-12-23,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"
3,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:578aa0fb-3e57-4b94-9c9e-9f31c668d2a6,2022-12-16,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"
4,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:794b25e2-b7fa-4066-8771-6e7b0bdee4f8,2022-12-23,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"
5,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:881e3825-87ca-4097-98ba-e0130020d65e,2022-11-30,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"
6,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:addf2ef3-993f-4c46-a362-3ce46a4d1da3,2022-12-08,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"
7,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:bf8fea4a-4f8d-4c65-b7b0-8bd055472389,2022-12-30,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"
8,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:c6d86888-b0aa-4f4b-83ff-59c5f0626123,2022-12-21,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"
9,urn:user:c543d399-134b-47d0-b3b1-d9a64c97912b,urn:order:ce24cad6-b0f7-4ebc-addd-9c6eaaca529e,2022-12-23,"[Start, consumer referral, direct, expert affiliate, google, google ads, google shopping, linkedin, trigger, Conversion]"


In [20]:
df_paths = df_paths.rename(columns={"fact_attributions_touch_source": "path"})
df_paths.shape

(11, 4)

In [21]:
# def is_conversion(x):
#     if 'Conversion' in x['path']:
#         return True
#     return False

# df_paths['conversion'] = df_paths.apply(lambda x: is_conversion(x), axis=1)

In [22]:
# df_paths = df_paths[df_paths['conversion']]
# df_paths.head()

In [23]:
# df_paths['pathstring'] = [' > '.join(map(str, l)) for l in df_paths.path]
# df_paths.head()
# new_df = pd.DataFrame({'Paths':df_paths['pathstring']})
# new_df.head()

In [24]:
# import markov_model_attribution as mma

# df = pd.DataFrame({'Paths':['start > cone > ctwo > cthree > conv',
#                            'start > cone > null',
#                            'start > ctwo > cthree > null']})

# model = mma.run_model(paths=new_df)

# print(model['markov_conversions'])
# print(model['last_touch_conversions'])

### Markov Chains

In [25]:
list_of_paths = df_paths['path']
total_conversions = sum(path.count('Conversion') for path in df_paths['path'].tolist())
base_conversion_rate = total_conversions / len(list_of_paths)

In [26]:
print(total_conversions)
print(base_conversion_rate)

10
0.9090909090909091


In [27]:
def transition_states(list_of_paths):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    transition_states = {x + '>' + y: 0 for x in list_of_unique_channels for y in list_of_unique_channels}

    for possible_state in list_of_unique_channels:
        if possible_state not in ['Conversion', 'Null']:
            for user_path in list_of_paths:
                if possible_state in user_path:
                    indices = [i for i, s in enumerate(user_path) if possible_state in s]
                    for col in indices:
                        transition_states[user_path[col] + '>' + user_path[col + 1]] += 1

    return transition_states


trans_states = transition_states(list_of_paths)

In [28]:
# trans_states

In [29]:
def transition_prob(trans_dict):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    trans_prob = defaultdict(dict)
    for state in list_of_unique_channels:
        if state not in ['Conversion', 'Null']:
            counter = 0
            index = [i for i, s in enumerate(trans_dict) if state + '>' in s]
            for col in index:
                if trans_dict[list(trans_dict)[col]] > 0:
                    counter += trans_dict[list(trans_dict)[col]]
            for col in index:
                if trans_dict[list(trans_dict)[col]] > 0:
                    state_prob = float((trans_dict[list(trans_dict)[col]])) / float(counter)
                    trans_prob[list(trans_dict)[col]] = state_prob

    return trans_prob


trans_prob = transition_prob(trans_states)

In [30]:
# trans_prob

In [31]:
def transition_matrix(list_of_paths, transition_probabilities):
    trans_matrix = pd.DataFrame()
    list_of_unique_channels = set(x for element in list_of_paths for x in element)

    for channel in list_of_unique_channels:
        trans_matrix[channel] = 0.00
        trans_matrix.loc[channel] = 0.00
        trans_matrix.loc[channel][channel] = 1.0 if channel in ['Conversion', 'Null'] else 0.0

    for key, value in transition_probabilities.items():
        origin, destination = key.split('>')
        trans_matrix.at[origin, destination] = value

    return trans_matrix

trans_matrix = transition_matrix(list_of_paths, trans_prob)

In [39]:
trans_matrix

,google,expert affiliate,google shopping,trigger,linkedin,Conversion,Null,direct,Start,consumer referral,google ads
google,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0
expert affiliate,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
google shopping,0.0,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
trigger,0.0,0.000000,0.0,0.000000,0.0,0.909091,0.090909,0.000000,0.0,0.000000,0.0
linkedin,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
Conversion,0.0,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0
Null,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0
direct,0.0,0.909091,0.0,0.090909,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
Start,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.090909,0.0,0.909091,0.0
consumer referral,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.0


In [33]:
def removal_effects(df, conversion_rate):
    removal_effects_dict = {}
    channels = [channel for channel in df.columns if channel not in ['Start',
                                                                     'Null',
                                                                     'Conversion']]
    for channel in channels:
        removal_df = df.drop(channel, axis=1).drop(channel, axis=0)
        for column in removal_df.columns:
            row_sum = np.sum(list(removal_df.loc[column]))
            null_pct = float(1) - row_sum
            if null_pct != 0:
                removal_df.loc[column]['Null'] = null_pct
            removal_df.loc['Null']['Null'] = 1.0

        removal_to_conv = removal_df[
            ['Null', 'Conversion']].drop(['Null', 'Conversion'], axis=0)
        removal_to_non_conv = removal_df.drop(
            ['Null', 'Conversion'], axis=1).drop(['Null', 'Conversion'], axis=0)

        removal_inv_diff = np.linalg.inv(
            np.identity(
                len(removal_to_non_conv.columns)) - np.asarray(removal_to_non_conv))
        removal_dot_prod = np.dot(removal_inv_diff, np.asarray(removal_to_conv))
        removal_cvr = pd.DataFrame(removal_dot_prod,
                                   index=removal_to_conv.index)[[1]].loc['Start'].values[0]
        removal_effect = 1 - removal_cvr / conversion_rate
        removal_effects_dict[channel] = removal_effect

    return removal_effects_dict


removal_effects_dict = removal_effects(trans_matrix, base_conversion_rate)

In [34]:
removal_effects_dict

{'google': 0.9090909090909091,
 'expert affiliate': 0.9090909090909091,
 'google shopping': 0.9090909090909091,
 'trigger': 1.0,
 'linkedin': 0.9090909090909091,
 'direct': 1.0,
 'consumer referral': 0.9090909090909091,
 'google ads': 0.9090909090909091}

In [35]:
pd.DataFrame({'re': removal_effects_dict}).sum()

re    7.454545
dtype: float64

In [36]:
# const = 50788/10453
def markov_chain_allocations(removal_effects, total_conversions):
    re_sum = np.sum(list(removal_effects.values()))
    print(re_sum)

    return {k: (v / re_sum) * total_conversions for k, v in removal_effects.items()}

attributions = markov_chain_allocations(removal_effects_dict, total_conversions)

7.454545454545455


In [37]:
attributions

{'google': 1.219512195121951,
 'expert affiliate': 1.219512195121951,
 'google shopping': 1.219512195121951,
 'trigger': 1.3414634146341462,
 'linkedin': 1.219512195121951,
 'direct': 1.3414634146341462,
 'consumer referral': 1.219512195121951,
 'google ads': 1.219512195121951}

In [38]:
pd.DataFrame({'att': attributions}).sum()

att    10.0
dtype: float64

In [1260]:
pd.DataFrame({'att': attributions}).to_csv('attributions.csv')